This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [115]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

D1odiVCbG5AKsZgWZxS_


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [116]:
# First, import the relevant modules
import requests
import statistics

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [117]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
params = {'api_key': API_KEY, 'start_date':'2020-12-01', 'end_date': '2020-12-01'}
r = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json', params)
print(r.url)

https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?api_key=D1odiVCbG5AKsZgWZxS_&start_date=2020-12-01&end_date=2020-12-01


In [118]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json())

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2020-12-01T14:48:09.907Z', 'newest_available_date': '2020-12-01', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2020-12-01', 'end_date': '2020-12-01', 'data': [['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None]], 'collapse': None, 'order': None, 'database_id': 6129}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [119]:
# 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, 
# for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
params = {'api_key': API_KEY, 'start_date':'2017-01-01', 'end_date': '2017-12-31'}
r = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json', params)
print('Requested URL:')
print(r.url)
print('Data Returned:')
print(r.json())

Requested URL:
https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?api_key=D1odiVCbG5AKsZgWZxS_&start_date=2017-01-01&end_date=2017-12-31
Data Returned:
{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2020-12-01T14:48:09.907Z', 'newest_available_date': '2020-12-01', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2017-01-01', 'end_date': '2017-12-31', 'data': [['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None], ['2017-12-28', 51

In [120]:
# 2. Convert the returned JSON object into a Python dictionary.
d = dict(r.json())
print(d.keys())
rows = d['dataset']['data']

dict_keys(['dataset'])


In [121]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.
# 4. What was the largest change in any one day (based on High and Low price)?
# 5. What was the largest change between any two days (based on Closing Price)?
# 6. What was the average daily trading volume during this year?
# 7. (Optional) What was the median trading volume during this year. 
# (Note: you may need to implement your own function for calculating the median.)
opening_low = float('inf')
opening_low_date = ''

opening_high = float('-inf')
opening_high_date = ''

largest_single_day_change_date = ''
largest_single_day_change = float('-inf')

largest_two_day_change = float('-inf')
largest_two_day_change_range = ''

closing_price_yesterday = -1
last_date = ''

total_trading_vol = 0

trading_vols = []

for row in rows:
    if row[1] is not None:
        if row[1] > opening_high:
            opening_high = row[1]
            opening_high_date = row[0]
        elif row[1] < opening_low:
            opening_low = row[1]
            opening_low_date = row[0]
  
    if row[2] is not None and row[3] is not None:
        if (row[2] - row[3]) > largest_single_day_change:
            largest_single_day_change_date = row[0]
            largest_single_day_change = row[2] - row[3]
    
    if row[4] is not None:
        if closing_price_yesterday > 0 and abs(row[4] - closing_price_yesterday) > largest_two_day_change:
            largest_two_day_change = abs(row[4] - closing_price_yesterday)
            largest_two_day_change_range = row[0] + ' - ' + last_date

        closing_price_yesterday = row[4]
        last_date = row[0]
    
    if row[6] is not None:
        total_trading_vol = total_trading_vol + row[6]
        trading_vols.append(row[6])

avg_trading_vol = int(total_trading_vol/len(rows))
print('Opening low was ' + str(opening_low) + ' on ' + opening_low_date)
print('Opening high was ' + str(opening_high) + ' on ' + opening_high_date)
print('Largest single day change was ' + str(largest_single_day_change) + ' on ' + largest_single_day_change_date)
print('Largest two day change was ' + str(largest_two_day_change) + ' between ' + largest_two_day_change_range)
print('Average trading volume is ' + str(avg_trading_vol))
print('Median trading volume is ' + str(statistics.median(trading_vols)))

Opening low was 34.0 on 2017-01-24
Opening high was 53.11 on 2017-12-14
Largest single day change was 2.8100000000000023 on 2017-05-11
Largest two day change was 2.559999999999995 between 2017-08-08 - 2017-08-09
Average trading volume is 89124
Median trading volume is 76286.0
